In [1]:
import pandas as pd
from datetime import timezone
import matplotlib.pyplot as plt
import requests
import json

In [2]:
sensors = pd.read_csv('../DATA/processed/sensors.csv', index_col=0)

In [3]:
sensors.sensor_type.value_counts()

SDS011    256
BME280    181
Name: sensor_type, dtype: int64

In [4]:
sensors.head()

,sensor_id,sensor_type,lat,lon
0,27418,SDS011,55.697960,37.535890
1,21332,SDS011,55.698000,37.498000
2,42870,SDS011,55.887121,37.265067
3,34535,SDS011,55.596000,37.538000
4,43811,SDS011,55.703017,37.459441


In [5]:
sds_sensors = sensors[sensors.sensor_type=='SDS011']
bme_sensors = sensors[sensors.sensor_type=='BME280']

In [6]:
sensors = (sds_sensors.assign(dummy=1)
    .merge(bme_sensors.assign(dummy=1),on='dummy')
    .query('lat_x==lat_y and lon_x==lon_y')
    .drop('dummy', axis=1))[['sensor_id_x', 'sensor_id_y', 'lat_x', 'lon_x']]
sensors = sensors.rename(columns={'sensor_id_x': 'sds_sensor',
                                  'sensor_id_y': 'bme_sensor',
                                  'lat_x': 'lat',
                                  'lon_x': 'lon'})
sensors

,sds_sensor,bme_sensor,lat,lon
882,43811,43812,55.703017,37.459441
1003,46280,46281,55.677867,37.456325
1186,33626,33627,55.874089,37.510001
1400,54815,54816,55.752000,37.884000
1561,45568,45569,55.625773,37.525268
...,...,...,...,...
45234,42920,42921,55.850951,37.348591
45261,41097,41098,55.563750,37.644380
45496,48427,48428,55.595953,37.598865
45647,35514,35515,55.650000,37.490000


In [7]:
press_meteo = pd.read_csv('../DATA/processed/dataset.csv', parse_dates=['date']).set_index('date').pres_meteo

In [8]:
press_meteo

date
2019-04-01 00:00:00+00:00     99271.5612
2019-04-01 00:05:00+00:00     99471.5442
2019-04-01 00:10:00+00:00     99471.5442
2019-04-01 00:15:00+00:00     99471.5442
2019-04-01 00:20:00+00:00     99471.5442
                                ...     
2021-01-18 17:40:00+00:00    100231.4796
2021-01-18 17:45:00+00:00    100231.4796
2021-01-18 17:50:00+00:00    100231.4796
2021-01-18 17:55:00+00:00    100231.4796
2021-01-18 18:00:00+00:00    100231.4796
Name: pres_meteo, Length: 189721, dtype: float64

https://ru.wikipedia.org/wiki/%D0%91%D0%B0%D1%80%D0%BE%D0%BC%D0%B5%D1%82%D1%80%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B0%D1%8F_%D1%81%D1%82%D1%83%D0%BF%D0%B5%D0%BD%D1%8C

In [9]:
def get_sealevel(bme_id):
    # see details 
    file_name = f'{bme_id}_bme280_sensor_.csv'
    bme_data = pd.read_csv('../DATA/raw/sensors/'+file_name, sep=';', parse_dates=['timestamp'])
    bme_data = bme_data.set_index('timestamp').resample('5T').mean()
    bme_data = bme_data.tz_localize(timezone.utc)
    bme_data['press_diff'] = (bme_data['pressure'] - press_meteo) / 1000
    bme_data['Q'] = 8000/(bme_data.pressure/1000) * (1 + 0.00366*bme_data.temperature)
    bme_data['delta_h'] = - bme_data.Q * bme_data.press_diff
    delta_h = bme_data.delta_h.median()
    h = delta_h + 125 # высота метеостанции над уровнем моря
    return h
    

In [10]:
sealevels = []
for index, row in sensors.iterrows():
    sealevels.append(get_sealevel(int(row['bme_sensor'])))

//home/anatoly/HDD/anaconda3/envs/mlforair/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [11]:
sensors['sealevel'] = sealevels

In [12]:
import time

surface_levels = []
for index, row in sensors.iterrows():
    lat, lon = row.lat, row.lon
    resp = requests.get(f'https://api.opentopodata.org/v1/srtm30m?locations={lat},{lon}')
    data = json.loads(resp.text)
    surface_levels.append(data['results'][0]['elevation'])
    time.sleep(1)


In [13]:
sensors['surface_alt'] = surface_levels

In [14]:
sensors['surface_level'] = sensors.sealevel - sensors.surface_alt

In [15]:
sensors

,sds_sensor,bme_sensor,lat,lon,sealevel,surface_alt,surface_level
882,43811,43812,55.703017,37.459441,198.271350,163.0,35.271350
1003,46280,46281,55.677867,37.456325,179.684930,176.0,3.684930
1186,33626,33627,55.874089,37.510001,167.960469,176.0,-8.039531
1400,54815,54816,55.752000,37.884000,236.385167,153.0,83.385167
1561,45568,45569,55.625773,37.525268,263.580603,230.0,33.580603
...,...,...,...,...,...,...,...
45234,42920,42921,55.850951,37.348591,193.647666,176.0,17.647666
45261,41097,41098,55.563750,37.644380,169.748722,175.0,-5.251278
45496,48427,48428,55.595953,37.598865,194.379092,172.0,22.379092
45647,35514,35515,55.650000,37.490000,222.588161,200.0,22.588161


In [16]:
sensors.to_csv('sensors.csv', index=False)